Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from spacy.tokenizer import Tokenizer
from sklearn.naive_bayes import ComplementNB
import spacy

### Load Competition Data

In [3]:
# You may need to change the path
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

train['description'] = [x.replace('\n', '') for x in train['description']]
test['description'] = [x.replace('\n', '') for x in test['description']]

train, val = train_test_split(train, test_size=.8, train_size=.2)

In [4]:
train.head()

,id,description,ratingCategory
3453,3164,"Hot wet oak, berries, Red Hots, and wet cornme...",1
1997,3099,This 20 year old expression of Tullibardine ha...,1
3699,338,The fifth in a series of Glenlivet Cellar Coll...,1
3762,743,With the quintessential traditional bourbon bo...,1
3673,2565,A release chosen by Douglas Maclean (and named...,1


In [5]:
val.head()

,id,description,ratingCategory
863,1711,Glenrothes is always this fascinating mix of t...,1
2791,4403,South Africa’s most established distillery now...,0
2006,363,"Full-flavored, confident, and very dynamic. Br...",1
2468,1280,"Rich amber in hue, sweet, and mellow with init...",1
2756,3853,Who could refuse a blend of single malts wholl...,0


### Define Pipeline Components

In [6]:
nlp = spacy.load('en_core_web_lg')

def tokenize(document):
    
    doc = nlp(document)
    
    return [token.lemma_.strip().lower() for token in doc
            if (token.is_stop != True) 
            and (token.is_punct != True) 
            and (token.like_num != True)
            and (token.is_currency != True)
           and (token.is_digit != True)]

In [7]:
vect = TfidfVectorizer(stop_words=spacy.lang.en.stop_words.STOP_WORDS, ngram_range=(1,2))
rfc = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', rfc)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [147]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

random_search = RandomizedSearchCV(pipe,parameters, cv=5, n_jobs=-2, verbose=1)
random_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-2)]: Done  50 out of  50 | elapsed:    3.4s finished
/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('vect',
                                              TfidfVectorizer(analyzer='word',
                                                              binary=False,
                                                              decode_error='strict',
                                                              dtype=<class 'numpy.float64'>,
                                                              encoding='utf-8',
                                                              input='content',
                                                              lowercase=True,
                                                              max_df=1.0,
                                                              max_features=None,
                                                              min_df=1,
                                                        

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [148]:
# Predictions on test sample
pred = random_search.predict(val['description'])
accuracy_score(val['ratingCategory'], pred)

0.7030581039755351

In [150]:
random_search.best_params_

{'vect__min_df': 0.02,
 'vect__max_features': 1000,
 'vect__max_df': 1.0,
 'clf__n_estimators': 10,
 'clf__max_depth': 15}

In [80]:
pred = grid_search.predict(test['description'])
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [81]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [7]:
subNumber = 8

In [83]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [8]:
from sklearn.decomposition import TruncatedSVD

In [9]:
vect = TfidfVectorizer(stop_words=spacy.lang.en.stop_words.STOP_WORDS, ngram_range=(1,2))
svd = TruncatedSVD()
clf = RandomForestClassifier()
lsi = Pipeline([('vect', vect), ('svd', svd)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [125]:
pipe = Pipeline([('lsi', lsi), ('clf', clf)])

parameters = {
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20),
    'lsi__vect__min_df': (.02, .05),
    'lsi__vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
}

random_search = RandomizedSearchCV(pipe, parameters, cv=5, n_jobs=-2, verbose=1)
random_search.fit(train['description'], train['ratingCategory'])

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-2)]: Done  50 out of  50 | elapsed:    2.1s finished
/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('lsi',
                                              Pipeline(memory=None,
                                                       steps=[('vect',
                                                               TfidfVectorizer(analyzer='word',
                                                                               binary=False,
                                                                               decode_error='strict',
                                                                               dtype=<class 'numpy.float64'>,
                                                                               encoding='utf-8',
                                                                               input='content',
                                                                               lowercase=True,
                          

### Make a Submission File

In [126]:
# Predictions on test sample
pred = random_search.predict(val['description'])
accuracy_score(val['ratingCategory'], pred)

0.7116207951070337

In [127]:
pred = grid_search.predict(test['description'])
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [128]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,0
1,2604,0
2,3341,1
3,3764,1
4,2306,1


In [129]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

In [249]:
train['ratingCategory'].value_counts(normalize=True)

1    0.687882
0    0.291310
2    0.020808
Name: ratingCategory, dtype: float64

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [10]:
def tokenize(document):
    
    doc = nlp(document)
    
    return [token.lemma_.strip().lower() for token in doc
            if (token.is_stop != True) 
            and (token.is_punct != True) 
            and (token.like_num != True)
            and (token.is_currency != True)
           and (token.is_digit != True)]

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint, uniform
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 

class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [11]:
# Apply to your Dataset



vect = TfidfVectorizer(stop_words=spacy.lang.en.stop_words.STOP_WORDS, ngram_range=(1,4), tokenizer=LemmaTokenizer())
svd = TruncatedSVD()
lsi = Pipeline([('vect', vect), ('svd', svd)])
# lsi = Pipeline([('svd', svd)])
# clf = GradientBoostingClassifier()
clf = LogisticRegression(multi_class='multinomial', n_jobs=-1)
pipe = Pipeline([('lsi', lsi), ('clf', clf)])

parameters = {
    'lsi__svd__n_components': randint(10, 250),
    'lsi__vect__max_df': uniform(0.75, 1.0),
    'lsi__vect__min_df': uniform(.02, .05),
    'lsi__vect__max_features': randint(500, 1000),
    'clf__penalty': ['l1', 'l2', 'elasticnet'],
    'clf__solver': ['newton-cg', 'sag', 'saga', 'lbfgs'],
    'clf__class_weight': [{0: .5, 1: .3, 2: .2}, 'balanced'],
    'clf__C': uniform(.1, 2)
}

In [270]:
random_search = RandomizedSearchCV(pipe, 
                                   parameters, 
                                   cv=10, 
                                   n_iter=200,
                                   n_jobs=-2, 
                                   verbose=True)
random_search.fit(train['description'], train['ratingCategory'])

Fitting 10 folds for each of 200 candidates, totalling 2000 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:   11.9s
[Parallel(n_jobs=-2)]: Done 186 tasks      | elapsed:   58.0s
[Parallel(n_jobs=-2)]: Done 436 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-2)]: Done 786 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-2)]: Done 1236 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-2)]: Done 1786 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-2)]: Done 2000 out of 2000 | elapsed: 10.1min finished
/opt/anaconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['d', 'doe', 'ha', 'le', 'll', 'm', 'n', 's', 't', 'u', 've', 'wa', '‘', '’'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('lsi',
                                              Pipeline(memory=None,
                                                       steps=[('vect',
                                                               TfidfVectorizer(analyzer='word',
                                                                               binary=False,
                                                                               decode_error='strict',
                                                                               dtype=<class 'numpy.float64'>,
                                                                               encoding='utf-8',
                                                                               input='content',
                                                                               lowercase=True,
                         

In [271]:
random_search.best_score_

0.7223276121650105

In [272]:
# Predictions on test sample
pred = random_search.predict(val['description'])
accuracy_score(val['ratingCategory'], pred)

0.6960244648318042

### Make a Submission File

In [273]:
# Predictions on test sample
pred = random_search.predict(test['description'])

In [274]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [275]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [276]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

In [11]:
# Continue Word Embedding Work Here
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

X = get_word_vectors(train['description'])

len(X) == len(train['description'])

True

In [12]:
X_val = get_word_vectors(val['description'])
X_test = get_word_vectors(test['description'])

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier

vect = TfidfVectorizer(stop_words=spacy.lang.en.stop_words.STOP_WORDS, tokenizer=LemmaTokenizer())
svd = TruncatedSVD()
# lsi = Pipeline([('vect', vect), ('svd', svd)])
lsi = Pipeline([('svd', svd)])
nn = KNeighborsClassifier(n_jobs=-1)
cnb = ComplementNB()

pipe = Pipeline([('lsi', lsi), ('nn', nn)])


parameters = {
    'lsi__svd__n_components': randint(2, 250),
    'lsi__svd__algorithm': ['randomized', 'arpack'],
#     'lsi__vect__max_df': uniform(0.5, 1.0),
#     'lsi__vect__min_df': uniform(.01, .1),
#     'lsi__vect__max_features': randint(300, 1000),
#     'lsi__vect__ngram_range': [(1,3), (2,4), (1,4), (2,3)],
    'nn__algorithm': ['ball_tree', 'kd_tree', 'brute'],
    'nn__weights': ['uniform', 'distance'],
    'nn__n_neighbors': randint(3,10),
    'nn__metric': ['minkowski', 'euclidean', 'mahalanobis']
}

In [26]:
random_search = RandomizedSearchCV(pipe, 
                                   parameters, 
                                   cv=10, 
                                   n_iter=1000,
                                   n_jobs=-2, 
                                   verbose=True)
random_search.fit(X, train['ratingCategory'])

Fitting 10 folds for each of 1000 candidates, totalling 10000 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-2)]: Done 186 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-2)]: Done 436 tasks      | elapsed:   36.0s
[Parallel(n_jobs=-2)]: Done 786 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-2)]: Done 1236 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-2)]: Done 1786 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-2)]: Done 2436 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-2)]: Done 3186 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-2)]: Done 4036 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-2)]: Done 4986 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-2)]: Done 6036 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-2)]: Done 7186 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-2)]: Done 8436 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-2)]: Done 9786 tasks      | elapsed: 12.8min
[Parallel(n_jobs=-2)]: Done 10000 out of 10000

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('lsi',
                                              Pipeline(memory=None,
                                                       steps=[('svd',
                                                               TruncatedSVD(algorithm='randomized',
                                                                            n_components=2,
                                                                            n_iter=5,
                                                                            random_state=None,
                                                                            tol=0.0))],
                                                       verbose=False)),
                                             ('nn',
                                              KNeighborsClassifier(algorithm='auto',
                                             

In [27]:
random_search.best_score_

0.7417494730502859

In [28]:
random_search.best_params_

{'lsi__svd__algorithm': 'randomized',
 'lsi__svd__n_components': 160,
 'nn__algorithm': 'ball_tree',
 'nn__metric': 'minkowski',
 'nn__n_neighbors': 7,
 'nn__weights': 'uniform'}

In [29]:
pred = random_search.predict(X_val)
accuracy_score(val['ratingCategory'], pred)

0.7073394495412844

In [30]:
pred = random_search.predict(X_test)
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')
submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?